<a href="https://colab.research.google.com/github/IlyaHomka/MSc-Projects/blob/main/Bin_Packing_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install qiskit

In [2]:
! pip install docplex

In [3]:
! pip install qiskit-aer

In [4]:
! pip install qiskit-algorithms

In [5]:
!pip install qiskit-optimization==0.6.1

In [6]:
!pip install cplex

# Bin Packing Problem

The bin packing problem is an optimization problem where it is necessaru to assigning $N$ items of differing sizes into the smallest number of bins each with capacity $Q$.

## 1. Problem statement

$$\underset{\chi, \xi}{minimize}\sum_{i=1}^m \chi_i$$

subject to:
$$\sum_{i=1}^m \xi_{ij} = 1 \qquad j=1,...,n$$
$$\sum_{j=1}^n w_{j}\xi_{ij} \le Q \chi_i \qquad i = 1, ..., m$$
$$\xi_{ij}\in  \{0,1\} \qquad i=1,..,m \qquad j=1,..,n$$
$$\chi_{i}\in  \{0,1\} \qquad i=1,..,m $$

- n is the number of items
- m is the number of bins
- $w_j$ is the j-th item weight
- $\chi_i$ is i-th bin
- $\xi_{ij}$ is the variable that represent if the item j is in the bin i.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from docplex.mp.model import Model

from qiskit_aer import Aer
from qiskit_algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_optimization.algorithms import CplexOptimizer, MinimumEigenOptimizer
from qiskit_optimization.algorithms.admm_optimizer import ADMMParameters, ADMMOptimizer
from qiskit_optimization import QuadraticProgram

from qiskit_optimization.converters import QuadraticProgramToQubo

#1.	From Integer Linear Programming (ILP) to Quadratic Unconstrained Binary Optimization (QUBO)

##Define the ILP formulation of the BPP. You can use Docplex or similar frameworks to do it.

In [2]:
np.random.seed(2)
n = 3 # number of bins
m = n # number of items
Q = 40 # max weight of a bin

wj = np.random.randint(1,Q,n) # Randomly picking the item weight

In [3]:
# Construct model using docplex
mdl = Model("BinPacking")

x = mdl.binary_var_list([f"x{i}" for i in range(n)]) # list of variables that represent the bins
y = mdl.binary_var_list([f"y{i//m},{i%m}" for i in range(n*m)]) # variables that represent the items on the specific bin

objective = mdl.sum([x[i] for i in range(n)])

mdl.minimize(objective)

for j in range(m):
    # First set of constraints: the items must be in any bin
    constraint0 = mdl.sum([y[i*m+j] for i in range(n)])
    mdl.add_constraint(constraint0 == 1, f"cons0,{j}")

for i in range(n):
    # Second set of constraints: weight constraints
    constraint1 = mdl.sum([wj[j] * y[i*m+j] for j in range(m)])
    mdl.add_constraint(constraint1 <= Q * x[i], f"cons1,{i}")

In [4]:
print(mdl.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: BinPacking

Minimize
 obj: x0 + x1 + x2
Subject To
 cons0,0: y0,0 + y1,0 + y2,0 = 1
 cons0,1: y0,1 + y1,1 + y2,1 = 1
 cons0,2: y0,2 + y1,2 + y2,2 = 1
 cons1,0: 16 y0,0 + 9 y0,1 + 23 y0,2 - 40 x0 <= 0
 cons1,1: 16 y1,0 + 9 y1,1 + 23 y1,2 - 40 x1 <= 0
 cons1,2: 16 y2,0 + 9 y2,1 + 23 y2,2 - 40 x2 <= 0

Bounds
 0 <= x0 <= 1
 0 <= x1 <= 1
 0 <= x2 <= 1
 0 <= y0,0 <= 1
 0 <= y0,1 <= 1
 0 <= y0,2 <= 1
 0 <= y1,0 <= 1
 0 <= y1,1 <= 1
 0 <= y1,2 <= 1
 0 <= y2,0 <= 1
 0 <= y2,1 <= 1
 0 <= y2,2 <= 1

Binaries
 x0 x1 x2 y0,0 y0,1 y0,2 y1,0 y1,1 y1,2 y2,0 y2,1 y2,2
End



##Test your function with specific instances (size small, medium, and big)



In [5]:
# Medium Sumber of Bins
n1 = 10 # number of bins
m1 = n1 # number of items
Q1 = 40 # max weight of a bin
wj1 = np.random.randint(1,Q1,n1) # Randomly picking the item weight

In [6]:
# Construct model using docplex
mdl1 = Model("BinPacking")

x1 = mdl1.binary_var_list([f"x{i}" for i in range(n1)]) # list of variables that represent the bins
y1 = mdl1.binary_var_list([f"y{i//m1},{i%m1}" for i in range(n1*m1)]) # variables that represent the items on the specific bin

objective1 = mdl1.sum([x1[i] for i in range(n1)])

mdl1.minimize(objective1)

for j in range(m1):
    # First set of constraints: the items must be in any bin
    constraint01 = mdl1.sum([y1[i*m1+j] for i in range(n1)])
    mdl1.add_constraint(constraint01 == 1, f"cons0,{j}")

for i in range(n):
    # Second set of constraints: weight constraints
    constraint11 = mdl1.sum([wj1[j] * y1[i*m1+j] for j in range(m1)])
    mdl1.add_constraint(constraint11 <= Q1 * x1[i], f"cons1,{i}")

In [7]:
# print(mdl1.export_as_lp_string())

In [8]:
# Large Number of Bins
n2 = 100 # number of bins
m2 = n2 # number of items
Q2 = 40 # max weight of a bin
wj2 = np.random.randint(1,Q2,n2) # Randomly picking the item weight

In [9]:
# Construct model using docplex
mdl2 = Model("BinPacking")

x2 = mdl2.binary_var_list([f"x{i}" for i in range(n2)]) # list of variables that represent the bins
y2 = mdl2.binary_var_list([f"y{i//m2},{i%m2}" for i in range(n2*m2)]) # variables that represent the items on the specific bin

objective2 = mdl2.sum([x2[i] for i in range(n2)])

mdl2.minimize(objective2)

for j in range(m2):
    # First set of constraints: the items must be in any bin
    constraint02 = mdl2.sum([y2[i*m2+j] for i in range(n2)])
    mdl2.add_constraint(constraint02 == 1, f"cons0,{j}")

for i in range(n):
    # Second set of constraints: weight constraints
    constraint12 = mdl2.sum([wj2[j] * y2[i*m2+j] for j in range(m2)])
    mdl2.add_constraint(constraint12 <= Q2 * x2[i], f"cons1,{i}")

In [10]:
# print(mdl2.export_as_lp_string())

##Create a function to transform the ILP model into a QUBO

In [11]:
# Converting the docplex model into a QUBO model
from qiskit_optimization.translators import from_docplex_mp

def docplex_to_qubo(my_mdl):
  qp = from_docplex_mp(my_mdl)
  conv = QuadraticProgramToQubo()
  model_qubo = conv.convert(qp)
  return model_qubo

In [12]:
small_model_qubo = docplex_to_qubo(mdl)
medium_model_qubo = docplex_to_qubo(mdl1)
large_model_qubo = docplex_to_qubo(mdl2)

#2.	Create a Brute Force solver for the QUBO problem and solve the specific instances.




In [13]:
from itertools import permutations
def brute_force_apporach(w, q):
    # Generate all permutations of items (weihghts)
    all_permutations = permutations(w)

    min_bins = float('inf')  # Start with a very large number of bins

    for perm in all_permutations:
        bins = []

        # Try to pack items in the current permutation
        for item in perm:
            packed = False
            # Try to place the item in existing bins
            for bin in bins:
                if sum(bin) + item <= q:
                    bin.append(item)
                    packed = True
                    break
            # If no bin could accommodate the item, start a new bin
            if not packed:
                bins.append([item])

        # Record the solution with the minimum number of bins used
        min_bins = min(min_bins, len(bins))

    return min_bins

# Solve using brute force
min_bins = brute_force_apporach(wj, Q)
print(f"Minimum bins required: {min_bins}")

Minimum bins required: 2


#3.	To solve the QUBO, use quantum annealing simulators

## Using Cplex

In [14]:
import cplex

In [15]:
result = CplexOptimizer().solve(small_model_qubo)

In [16]:
print(f"Minimum bins required: {result.fval}")

Minimum bins required: 2.0


https://github.com/qiskit-community/qiskit-optimization/blob/main/docs/tutorials/02_converters_for_quadratic_programs.ipynb

https://github.com/qiskit-community/qiskit-optimization/blob/stable/0.5/docs/tutorials/06_examples_max_cut_and_tsp.ipynb

https://chatgpt.com/c/670bf488-c88c-8002-a0b4-6ff073ed0038


# 4. Use a Quantum Variational approach to solve the QUBO.

In [17]:
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler, Estimator

In [18]:
# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [19]:
backend = Aer.get_backend('qasm_simulator')
# Define the VQE optimizer
optimizer = COBYLA(maxiter=20)
estimator = Estimator();

from qiskit.circuit.library import TwoLocal
# Define the variational ansatz (parameterized quantum circuit) for 3 qubits
ansatz = TwoLocal(rotation_blocks='ry', entanglement_blocks='cz', reps=3, entanglement='linear')

In [20]:
from qiskit_algorithms import VQE

# Set up the VQE algorithm
vqe = VQE(estimator=estimator, ansatz=ansatz, optimizer=optimizer) #

#vqe = MinimumEigenOptimizer(VQE(estimator=estimator, ansatz=ansatz, optimizer=optimizer))

In [ ]:
# Solve the problem using VQE
vqe_optimizer = MinimumEigenOptimizer(vqe)
result_vqe = vqe_optimizer.solve(small_model_qubo)

In [ ]:
# Solve the problem using VQE
result_vqe = vqe.solve(small_model_qubo)
# Output the results
print("Optimal solution:", result_vqe.x)
print("Optimal value:", result_vqe.fval)

#5.	Use QAOA to solve the QUBO.

In [24]:
# Solve the problem using QAOA
sampler = Sampler()
qaoa = MinimumEigenOptimizer(QAOA(reps=3, optimizer=optimizer, sampler = sampler))

In [ ]:
result_qaoa = qaoa.solve(small_model_qubo)
print(result_qaoa)

# Output the results
print("Optimal value:", result_qaoa.fval)

#6.	Compare and analyze the results.

Since VQE and QAOA consume a lot of memory, they do not work on my Google Colab, but quantum annealing is the fastest one! Actually, I demosntrated teh same in my publication becasue VQE and QAOA compute kernels, which consumes a  significant amount of RAM!